> 이 파일은 커스텀 학습된 llm 모델을 허깅페이스를 통해 로드하고 테스트하는 용도의 파일.

In [1]:
import huggingface_hub
huggingface_hub.login() # 허깅페이스 억세스 토큰 필요

In [4]:
!pip install transformers

In [4]:
import transformers
from transformers import AutoTokenizer, pipeline

import torch

model = "taeseo06/llama2-PersonalizedFinetuning"

tokenizer = AutoTokenizer.from_pretrained(model)

pipe = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    # device_map="auto",
)

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.84G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [25]:
model = model.float()

AttributeError: 'str' object has no attribute 'float'

In [24]:
from transformers import LlamaForCausalLM, LlamaTokenizer

base_model = "taeseo06/llama2-PersonalizedFinetuning"
device_map = 'auto'

model = LlamaForCausalLM.from_pretrained(
        base_model,
        # load_in_8bit=True,
        torch_dtype=torch.float16,
        device_map=device_map,
    )

model = model.float()

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [13]:
!pip install accelerate
pipe2 = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [18]:
torch.set_default_tensor_type(torch.cuda.HalfTensor)

/usr/local/lib/python3.10/dist-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [20]:
pipe("너는 누구야?")

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [17]:
print(pipe("I'm a Korean. and love BTS",
                max_length=30,
                num_return_sequences=2))

RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

In [8]:
# sequences = pipe(
#     '너는 누구야?\n',
#     do_sample=True,
#     top_k=10,
#     num_return_sequences=1,
#     device_map="auto",
# )



sequences = pipe(
    '너는 누구야?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1
)


RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

In [6]:
print(pipe("넌 누구야?"))

RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

In [5]:
sequences = pipe(
    '너는 누구야?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)

RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

In [ ]:
sequences = pipeline(
    'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")